### Dépendances

In [1]:
import pandas as pd

# Dépendances traitement de texte
import spacy
import nltk
nlp = spacy.load("fr_core_news_sm")
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=stopwords.words('french')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\proui\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
body=pd.read_csv('data.csv',index_col=[0])  #éviter 'Unnamed: 0'
body.head()

,Nom,Mouvement,summary,sum_lem,mvm_label
0,William S. Burroughs,Beat Generation,William Seward Burroughs (/ˈwɪljəm ˈsuɚd ˈbɜɹo...,William Seward Burroughs ˈsuɚd dit William Bur...,0
1,Lucien Carr,Beat Generation,"Lucien Carr, né le 1er mars 1925 à New York et...",Lucien Carr naître premier mars 1925 New York ...,0
2,Carolyn Cassady,Beat Generation,"Carolyn Cassady, née Carolyn Robinson le 28 av...",Carolyn Cassady naître Carolyn Robinson 28 avr...,0
3,John Clellon Holmes,Beat Generation,"John Clellon Holmes est un romancier, essayist...",John Clellon holme romancier essayiste poète a...,0
4,Gregory Corso,Beat Generation,"Gregory Nunzio Corso (New York, 26 mars 1930 -...",gregory Nunzio Corso New York 26 mars 1930 Min...,0


In [3]:
print('On a '+str(max(body['mvm_label']))+' mouvements différents dans le dataset.')

On a 38 mouvements différents dans le dataset.


### Transformations

In [5]:
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import CountVectorizer #1
from sklearn.feature_extraction.text import TfidfVectorizer #2

> Représentation

In [35]:
corpus = body['sum_lem']

# == Choix du Vectorizer ==
vectorizer=TfidfVectorizer()
#vectorizer=CountVectorizer(ngram_range=(1,1))

sparse_matrix=vectorizer.fit_transform(corpus)      #apprentissage du vocabulaire (vectorizer affecté)
#vectorizer.get_feature_names_out()                 #longueur = nb de termes uniques présents dans l'ensemble des docs

In [47]:
xtrain, xtest, ytrain, ytest = train_test_split(body['sum_lem'],body['mvm_label'],shuffle=True, test_size = 0.35)

xtrain=vectorizer.transform(xtrain)
xtest=vectorizer.transform(xtest)

### Modèles

On va tester différents modèles connus.

#### RandomForest

In [60]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=150)
rfc.fit(xtrain, ytrain)
rfc.score(xtest,ytest)

0.4036697247706422

#### Naive Bayes

In [63]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(xtrain, ytrain)
mnb.score(xtest,ytest)

0.11926605504587157

#### SVM

In [62]:
from sklearn import svm
svmod=svm.SVC()
svmod.fit(xtrain,ytrain)
svmod.score(xtest,ytest)

0.12385321100917432